# MBX usage

In [1]:
import quippy
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import StringIO
import glob
import sys
import ase

In [2]:
this_dir = os.getcwd()

In [3]:

root_mbx_functions=(this_dir.split("tutorials")[0])
sys.path.insert(0,root_mbx_functions)
import mbx_functions
import unwrap_oh2_ch4

In [4]:
from CalcMBXnonperiodic import CalcMBX
potMBX_non_periodic = CalcMBX()

In [5]:
from SuperCellCalcMBX import SuperCellCalcMBX
potMBX_periodic = SuperCellCalcMBX()

### one can check the possible arguments of the potential assignment as:

In [6]:
SuperCellCalcMBX?

```Init signature: SuperCellCalcMBX(self, min_box_requirement=24.0, my_json='mbx_pbc.json', **kwargs)
```

one can specify to use another json file in which different polynomials can be turned off (to use the classical force field's terms instead)



an example json file:
```{
   "Note" : "This is a cofiguration file",
   "MBX" : {
       "box" : [50.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,50.0],
       "twobody_cutoff"   : 9.0,
       "threebody_cutoff" : 7.0,
       "max_n_eval_1b"    : 500,
       "max_n_eval_2b"    : 500,
       "max_n_eval_3b"    : 500,
       "dipole_tolerance" : 1E-16,
       "dipole_max_it"    : 100,
       "dipole_method"     : "cg",
       "alpha_ewald_elec" : 0.6,
       "grid_density_elec" : 2.5,
       "spline_order_elec" : 6,
       "alpha_ewald_disp" : 0.6,
       "grid_density_disp" : 2.5,
       "spline_order_disp" : 6,
       "ttm_pairs" : [],
       "ignore_2b_poly" : [],
       "ignore_3b_poly" : []
   } ,
   "i-pi" : {
       "port" : 34543,
       "localhost" : "localhost"
   }
}
```

If modifying the json files, one can leave most of the parameters as they are (the lattice will be set to the structures lattice in QUIP, it's just needed so that MBX knows that it's a periodic structure).

Certain PIPs can be turned off by modifying the following lines:
```
       "ignore_2b_poly" : [],
       "ignore_3b_poly" : []
```
to e.g.:
```
       "ignore_2b_poly" : [],
       "ignore_3b_poly" : [["ch4","h2o","h2o"]]
```
in case we would like to test the version without the methane-water-water 3B fit.

### Calculations with the above defined potentials:

#### Non-periodic calls:

In [7]:
## read in the Atoms object:
at_m1w5 = ase.io.read("m1w5_fixed.xyz")

#### make sure the structure's atoms are ordered as OHHOHH...CHHHHCHHHH..OHHOHH..

In [9]:
at_m1w5.symbols

Symbols('CH4OH2OH2OH2OH2OH2')

In [10]:
## assign the calculator to the Atoms object:
at_m1w5.set_calculator(potMBX_non_periodic)

In [11]:
## perform energy calculation:
e = at_m1w5.get_potential_energy()
print e

-0.4057065229


#### Periodic calls

In [12]:
## read in the Atoms object:
at_clathrate = ase.io.read("phase_clathrate.xyz")

In [13]:
at_clathrate.symbols

Symbols('OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2CH4CH4CH4CH4CH4CH4CH4CH4')

In [14]:
## assign the calculator to the Atoms object:
at_clathrate.set_calculator(potMBX_periodic)

In [15]:
## perform energy calculation:
e = at_clathrate.get_potential_energy()
print e

-29.0782165281


### one can look at the atoms object in ase:

In [16]:
from ase.visualize import view

In [17]:
view(at_m1w5)

In [18]:
view(at_clathrate)

## The following functions are only needed if wanting to call the potential with the quip command line commands

### "Unwrapping" structures
The ase Calculator object takes care of "unwrapping" the structure
but in case we'd like to use MBX without the ase helper functions,
we need an input where the molecules are not cut into halves at the pbc.

In [19]:
at_clathrate_unwrapped_dict = unwrap_oh2_ch4.unwrap_water_methane(at_clathrate)

In [20]:
at_clathrate_unwrapped_dict

{'phase_m': Atoms(symbols='CH4CH4CH4CH4CH4CH4CH4CH4', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...),
 'phase_mw': Atoms(symbols='C8H124O46', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...),
 'phase_w': Atoms(symbols='H92O46', pbc=True, cell=[11.7429, 11.7429, 11.7429], map_shift=..., n_neighb=...)}

In [21]:
at_clathrate_unwrapped = at_clathrate_unwrapped_dict["phase_mw"]

In [22]:
view(at_clathrate_unwrapped)

In [10]:
ase.io.write("phase_clathrate_unwrapped.xyz", at_clathrate_unwrapped)

### Building supercells
MBX needs the Atoms object to have at least 24. A boxlengths.
The ase Calculator also builds the supercells of the Atoms object but if needed, we can create an input as:

#### Don't forget to firstly unwrap the cell!
(The supercell would work but MBX needs the atoms to be ordered according to the molecules)

In [11]:
at_clathrate_unwrapped.cell

array([[11.7429,  0.    ,  0.    ],
       [ 0.    , 11.7429,  0.    ],
       [ 0.    ,  0.    , 11.7429]])

In [12]:
min_box_length=24.

In [13]:
lattice = at_clathrate_unwrapped.cell
s_i_list =[1,1,1]
for i in range(3):
    lat_ii=lattice[i][i]
    s_i_list[i]= (int(np.ceil(min_box_length/lat_ii)))
print s_i_list

[3, 3, 3]


In [14]:
at_super = at_clathrate_unwrapped.copy()*(s_i_list)

In [15]:
len(at_super)/len(at_clathrate_unwrapped)

27

In [16]:
ase.io.write("phase_clathrate_unwrapped_supercell.xyz", at_super)

### one can also generate strings needed for potential calls without the python interface:

In [21]:
mbx_functions.create_mbx_potential_variables(at_clathrate_unwrapped)

{'n_monomers_types': '46_OHH_8_CHHHH', 'nmon': 54}

In [17]:
mbx_functions.create_mbx_potential_variables(at_super)

{'n_monomers_types': '46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH_46_OHH_8_CHHHH',
 'nmon': 1458}

In [19]:
## number of atoms (needed for lammps):
len(at_super)

4806

In [22]:
len(at_clathrate_unwrapped)

178

Note: I used a newer version of ase to convert these xyz-s to the lammps's datafile:
```
import ase                                                                                                                                                                                          

from ase.io import lammpsdata                                                                                                                                                                       

at_in=ase.io.read("/home/es732/mnt/womble0/test_lammps/small_system/phase_clathrate_unwrapped.xyz")                                                                                                 
lammpsdata.write_lammps_data("/home/es732/mnt/womble0/test_lammps/small_system/data.mw",at_in)                                                                                                      

at_in=ase.io.read("/home/es732/mnt/womble0/test_lammps/supercell//phase_clathrate_unwrapped_supercell.xyz")                                                                                         
lammpsdata.write_lammps_data("/home/es732/mnt/womble0/test_lammps/supercell//data.mw",at_in)                 
```